In [1]:
import pandas as pd
import time
import numpy as np
from sklearn.feature_selection import chi2
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
import pickle
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")

In [2]:
# Import Dataset
dataset = pd.read_csv('user_behavior_dataset.csv')

In [3]:
# One-hot Encoading or Label Encoading

map_model = {'Google Pixel 5':0,'OnePlus 9':1,'Xiaomi Mi 11':2,'Samsung Galaxy S21':3,'iPhone 12':4}
map_Operating_System = {'Android':0,'iOS':1}
map_gender = {'Female':0,'Male':1}
dataset['Device Model'] = dataset['Device Model'].replace(map_model)
dataset['Operating System'] = dataset['Operating System'].replace(map_Operating_System)
dataset['Gender'] = dataset['Gender'].replace(map_gender)

dataset

,User ID,Device Model,Operating System,App Usage Time (min/day),Screen On Time (hours/day),Battery Drain (mAh/day),Number of Apps Installed,Data Usage (MB/day),Age,Gender,User Behavior Class
0,1,0,0,393,6.4,1872,67,1122,40,1,4
1,2,1,0,268,4.7,1331,42,944,47,0,3
2,3,2,0,154,4.0,761,32,322,42,1,2
3,4,0,0,239,4.8,1676,56,871,20,1,3
4,5,4,1,187,4.3,1367,58,988,31,0,3
...,...,...,...,...,...,...,...,...,...,...,...
695,696,4,1,92,3.9,1082,26,381,22,1,2
696,697,2,0,316,6.8,1965,68,1201,59,1,4
697,698,0,0,99,3.1,942,22,457,50,0,2
698,699,3,0,62,1.7,431,13,224,44,1,1


In [4]:
dataset.drop(['User ID'],axis=1,inplace=True)

In [5]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 700 entries, 0 to 699
Data columns (total 10 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Device Model                700 non-null    int64  
 1   Operating System            700 non-null    int64  
 2   App Usage Time (min/day)    700 non-null    int64  
 3   Screen On Time (hours/day)  700 non-null    float64
 4   Battery Drain (mAh/day)     700 non-null    int64  
 5   Number of Apps Installed    700 non-null    int64  
 6   Data Usage (MB/day)         700 non-null    int64  
 7   Age                         700 non-null    int64  
 8   Gender                      700 non-null    int64  
 9   User Behavior Class         700 non-null    int64  
dtypes: float64(1), int64(9)
memory usage: 54.8 KB


In [6]:
# input & output data spliting

input = dataset[['Device Model', 'Operating System',
       'App Usage Time (min/day)', 'Screen On Time (hours/day)',
       'Battery Drain (mAh/day)', 'Number of Apps Installed',
       'Data Usage (MB/day)', 'Age', 'Gender']]
output = dataset[['User Behavior Class']]

In [7]:
from sklearn.feature_selection import SelectKBest
def selectkbest(input,output,n):
        test = SelectKBest(score_func=chi2, k=n)
        fit1= test.fit(input,output)
        selectk_features = fit1.transform(input)
        return selectk_features

In [8]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
def split_scalar(inp,out):
        X_train, X_test, y_train, y_test = train_test_split(inp, out, test_size = 0.25, random_state = 0)
        sc = StandardScaler()
        X_train = sc.fit_transform(X_train)
        X_test = sc.transform(X_test)    
        return X_train, X_test, y_train, y_test

In [9]:
def logistic(X_train,y_train,X_test):       
        # Fitting K-NN to the Training set
        from sklearn.linear_model import LogisticRegression
        classifier = LogisticRegression(random_state = 0)
        classifier.fit(X_train, y_train)
        classifier,Accuracy,report,X_test,y_test,cm=cm_prediction(classifier,X_test)
        return  classifier,Accuracy,report,X_test,y_test,cm      

In [10]:
def cm_prediction(classifier,X_test):
     y_pred = classifier.predict(X_test)
        
        # Making the Confusion Matrix
     from sklearn.metrics import confusion_matrix
     cm = confusion_matrix(y_test, y_pred)
        
     from sklearn.metrics import accuracy_score 
     from sklearn.metrics import classification_report 
        #from sklearn.metrics import confusion_matrix
        #cm = confusion_matrix(y_test, y_pred)
        
     Accuracy=accuracy_score(y_test, y_pred )
        
     report=classification_report(y_test, y_pred)
     return  classifier,Accuracy,report,X_test,y_test,cm

In [11]:
def svm_linear(X_train,y_train,X_test):
                
        from sklearn.svm import SVC
        classifier = SVC(kernel = 'linear', random_state = 0)
        classifier.fit(X_train, y_train)
        classifier,Accuracy,report,X_test,y_test,cm=cm_prediction(classifier,X_test)
        return  classifier,Accuracy,report,X_test,y_test,cm

In [12]:
def svm_NL(X_train,y_train,X_test):
                
        from sklearn.svm import SVC
        classifier = SVC(kernel = 'rbf', random_state = 0)
        classifier.fit(X_train, y_train)
        classifier,Accuracy,report,X_test,y_test,cm=cm_prediction(classifier,X_test)
        return  classifier,Accuracy,report,X_test,y_test,cm

In [13]:
def Navie(X_train,y_train,X_test):       
        # Fitting K-NN to the Training set
        from sklearn.naive_bayes import GaussianNB
        classifier = GaussianNB()
        classifier.fit(X_train, y_train)
        classifier,Accuracy,report,X_test,y_test,cm=cm_prediction(classifier,X_test)
        return  classifier,Accuracy,report,X_test,y_test,cm         

In [14]:
def knn(X_train,y_train,X_test):
           
        # Fitting K-NN to the Training set
        from sklearn.neighbors import KNeighborsClassifier
        classifier = KNeighborsClassifier(n_neighbors = 5, metric = 'minkowski', p = 2)
        classifier.fit(X_train, y_train)
        classifier,Accuracy,report,X_test,y_test,cm=cm_prediction(classifier,X_test)
        return  classifier,Accuracy,report,X_test,y_test,cm

In [15]:
def Decision(X_train,y_train,X_test):
        
        # Fitting K-NN to the Training set
        from sklearn.tree import DecisionTreeClassifier
        classifier = DecisionTreeClassifier(criterion = 'entropy', random_state = 0)
        classifier.fit(X_train, y_train)
        classifier,Accuracy,report,X_test,y_test,cm=cm_prediction(classifier,X_test)
        return  classifier,Accuracy,report,X_test,y_test,cm      

In [16]:
def random(X_train,y_train,X_test):
        
        # Fitting K-NN to the Training set
        from sklearn.ensemble import RandomForestClassifier
        classifier = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = 0)
        classifier.fit(X_train, y_train)
        classifier,Accuracy,report,X_test,y_test,cm=cm_prediction(classifier,X_test)
        return  classifier,Accuracy,report,X_test,y_test,cm

In [17]:
def selectk_Classification(acclog,accsvml,accsvmnl,accknn,accnav,accdes,accrf): 
    
    dataframe=pd.DataFrame(index=['ChiSquare'],columns=['Logistic','SVMl','SVMnl','KNN','Navie','Decision','Random'])
    for number,idex in enumerate(dataframe.index):      
        dataframe['Logistic'][idex]=acclog[number]       
        dataframe['SVMl'][idex]=accsvml[number]
        dataframe['SVMnl'][idex]=accsvmnl[number]
        dataframe['KNN'][idex]=accknn[number]
        dataframe['Navie'][idex]=accnav[number]
        dataframe['Decision'][idex]=accdes[number]
        dataframe['Random'][idex]=accrf[number]
    return dataframe

In [18]:
kbest=selectkbest(input,output,9)       

acclog=[]
accsvml=[]
accsvmnl=[]
accknn=[]
accnav=[]
accdes=[]
accrf=[]

In [19]:
kbest

array([[0.000e+00, 0.000e+00, 3.930e+02, ..., 1.122e+03, 4.000e+01,
        1.000e+00],
       [1.000e+00, 0.000e+00, 2.680e+02, ..., 9.440e+02, 4.700e+01,
        0.000e+00],
       [2.000e+00, 0.000e+00, 1.540e+02, ..., 3.220e+02, 4.200e+01,
        1.000e+00],
       ...,
       [0.000e+00, 0.000e+00, 9.900e+01, ..., 4.570e+02, 5.000e+01,
        0.000e+00],
       [3.000e+00, 0.000e+00, 6.200e+01, ..., 2.240e+02, 4.400e+01,
        1.000e+00],
       [1.000e+00, 0.000e+00, 2.120e+02, ..., 8.280e+02, 2.300e+01,
        0.000e+00]])

In [20]:
X_train, X_test, y_train, y_test=split_scalar(kbest,output)   
    
        
classifier,Accuracy,report,X_test,y_test,cm=logistic(X_train,y_train,X_test)
acclog.append(Accuracy)

classifier,Accuracy,report,X_test,y_test,cm=svm_linear(X_train,y_train,X_test)  
accsvml.append(Accuracy)
    
classifier,Accuracy,report,X_test,y_test,cm=svm_NL(X_train,y_train,X_test)  
accsvmnl.append(Accuracy)
    
classifier,Accuracy,report,X_test,y_test,cm=knn(X_train,y_train,X_test)  
accknn.append(Accuracy)
    
classifier,Accuracy,report,X_test,y_test,cm=Navie(X_train,y_train,X_test)  
accnav.append(Accuracy)
    
classifier,Accuracy,report,X_test,y_test,cm=Decision(X_train,y_train,X_test)  
accdes.append(Accuracy)
    
classifier,Accuracy,report,X_test,y_test,cm=random(X_train,y_train,X_test)  
accrf.append(Accuracy)
    
result=selectk_Classification(acclog,accsvml,accsvmnl,accknn,accnav,accdes,accrf)

In [21]:
result

,Logistic,SVMl,SVMnl,KNN,Navie,Decision,Random
ChiSquare,1.0,1.0,1.0,0.982857,1.0,1.0,1.0


In [22]:
# Save the best model

import pickle
filename = "Capstone_project.sav"

In [23]:
pickle.dump(classifier,open(filename,'wb'))

In [24]:
# load the saved model for test

Load_model=pickle.load(open("Capstone_project.sav",'rb'))
Result=Load_model.predict([[1,1,150,6.0,1000,20,1000,21,1]])
print("User Behavior Class: ",Result[0])

User Behavior Class:  5
